In [1]:
#
# Simple example of a Julia linear pogramming model
#
 
using LinearAlgebra

const MPS_EXAMPLE::String = """
NAME          EXAMPLE
ROWS
 N  COST
 L  C1
 L  C2
 L  C3
COLUMNS
    x1        COST       3
    x1        C1         1
    x1        C2         1
    x2        COST       2
    x2        C1         1
    x2        C3         1
RHS
    RHS       C1         4
    RHS       C2         2
    RHS       C3         3
BOUNDS
 UP BND       x1         2
 UP BND       x2         3
ENDATA
"""

"NAME          EXAMPLE\nROWS\n N  COST\n L  C1\n L  C2\n L  C3\nCOLUMNS\n    x1        COST       3\n    x1        C1         1\n    x1        C2         1\n    x2        COST       2\n    x2        C1         1\n    x2        C3         1\nRHS\n    RHS       C1         4\n    RHS       C2         2\n    RHS       C3         3\nBOUNDS\n UP BND       x1         2\n UP BND       x2         3\nENDATA\n"

In [2]:
# Define a struct to represent a Linear Programming problem
struct LPProblem
    c::Vector{Float64}  # Objective function coefficients
    A::Matrix{Float64}  # Constraint matrix
    b::Vector{Float64}  # Right-hand side of constraints
    vars::Vector{String}  # Variable names
end

In [3]:
function read_mps_from_string(mps_string::String)
    # Split the input string into lines
    lines = split(mps_string, '\n')
    
    # Initialize data structures to store MPS file sections
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => Dict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""

    # Parse the MPS file line by line
    for line in lines
        words = split(line)
        isempty(words) && continue

        # Check if this line is a section header
        if (line[1] != ' ') && words[1] in ["NAME", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        # Process each section according to MPS format
        if current_section == "NAME"
            # NAME section: Problem name
            sections["NAME"] = words[1]
        elseif current_section == "ROWS"
            # ROWS section: Constraint types and names
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name  # Identify the objective function
            end
        elseif current_section == "COLUMNS"
            # COLUMNS section: Variable coefficients
            col_name, row_name, value = words
            value = parse(Float64, value)
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = Dict()
            end
            sections["COLUMNS"][col_name][row_name] = value
        elseif current_section == "RHS"
            # RHS section: Right-hand side values
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            # BOUNDS section: Variable bounds
            bound_type, _, var_name, value = words
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
        end
    end

    # Convert parsed MPS data to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    # Initialize problem components
    c = zeros(n_vars)  # Objective function coefficients
    A = zeros(n_constraints, n_vars)  # Constraint matrix
    b = zeros(n_constraints)  # Right-hand side values

    # Populate objective function and constraint matrix
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
        constraint_index = 0
        for row in sections["ROWS"]
            if row.type != "N"
                constraint_index += 1
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
        end
    end

    # Populate right-hand side values
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
        end
    end

    # Process bound constraints
    bound_constraints = []
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            if haskey(sections["BOUNDS"][var], "LO")
                push!(bound_constraints, (i, 1, sections["BOUNDS"][var]["LO"]))
            end
            if haskey(sections["BOUNDS"][var], "UP")
                push!(bound_constraints, (i, -1, -sections["BOUNDS"][var]["UP"]))
            end
        end
    end

    # Extend A and b with bound constraints
    if !isempty(bound_constraints)
        n_bound_constraints = length(bound_constraints)
        A_extended = zeros(n_constraints + n_bound_constraints, n_vars)
        A_extended[1:n_constraints, :] = A
        b_extended = zeros(n_constraints + n_bound_constraints)
        b_extended[1:n_constraints] = b

        for (idx, (var_idx, coef, rhs)) in enumerate(bound_constraints)
            A_extended[n_constraints + idx, var_idx] = coef
            b_extended[n_constraints + idx] = rhs
        end

        A = A_extended
        b = b_extended
    end

    return LPProblem(c, A, b, vars)
end

read_mps_from_string (generic function with 1 method)

In [4]:
#=
Theoretical basis for each step:

Problem Formulation:

The Revised Simplex Method solves linear programming problems in the standard form:
Maximize c^T x
Subject to Ax = b
x ≥ 0


Slack Variables (Step 0):

Slack variables are added to convert inequality constraints to equality constraints.
This ensures that the problem is in standard form for the simplex method.


Basic and Non-Basic Variables:

The method partitions variables into basic (B) and non-basic (N) variables.
Initially, slack variables form the basis, corresponding to the identity matrix in the augmented constraint matrix.


Revised Simplex Iteration:
a. Basic Solution (Step 1):

Compute x_B = B^(-1) * b, where B is the basis matrix.
This gives the values of basic variables in the current solution.

b. Reduced Costs (Step 2):

Compute y = c_B' * B^(-1), where c_B are the objective coefficients of basic variables.
Calculate reduced costs: c_N - y' * A_N, where A_N are columns of non-basic variables.
Reduced costs measure the rate of change in the objective for unit increase in non-basic variables.

c. Optimality Check (Step 3):

If all reduced costs are non-positive, the current solution is optimal.
This is based on the theorem that a basic feasible solution is optimal if and only if all reduced costs are non-positive.

d. Entering Variable (Step 4):

Choose the non-basic variable with the most positive reduced cost to enter the basis.
This variable has the potential to improve the objective value the most.

e. Direction Computation (Step 5):

Compute d = B^(-1) * A_q, where A_q is the column of the entering variable.
This determines how basic variables change as the entering variable increases.

f. Unboundedness Check (Step 6):

If all elements of d are non-positive and the reduced cost is positive, the problem is unbounded.
This means we can increase the entering variable indefinitely, improving the objective without bound.

g. Leaving Variable (Step 7):

Perform the minimum ratio test: min(x_B_i / d_i) for d_i > 0.
This determines how far we can move along the edge without violating non-negativity constraints.

h. Basis Update (Step 8):

Swap the entering and leaving variables in the basis.
This moves to an adjacent extreme point of the feasible region.


Convergence:

The algorithm repeats these steps until optimality is reached or unboundedness is detected.
Each iteration either improves the objective value or determines that the problem is unbounded.
For non-degenerate problems, the algorithm is guaranteed to terminate in a finite number of steps.


Numerical Considerations:

Small tolerance (1e-10) is used for numerical stability in comparisons.
A maximum iteration limit prevents infinite loops in case of cycling (rare in practice, but possible in degenerate problems).

This implementation of the Revised Simplex Method efficiently solves linear programming problems by working with the inverse of the basis matrix and reduced costs, rather than maintaining the entire tableau as in the standard Simplex Method.
=#

function revised_simplex(lp::LPProblem)
    c, A, b = lp.c, lp.A, lp.b
    m, n = size(A)
    
    println("Initial problem:")
    println("c = ", c)
    println("A = ", A)
    println("b = ", b)
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A I(m)]  # Augment A with identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("Initial basis: ", B)
    println("Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ b  # Solve B * x_B = b for basic variables
        
        println("Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = zeros(n)
            x[B] = x_B
            println("Optimal solution found")
            return x[1:length(lp.vars)], dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ A[:, q]
        
        println("Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("New basis: ", B)
        println("New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

revised_simplex (generic function with 1 method)

In [8]:
# Example usage
lp = read_mps_from_string(MPS_EXAMPLE)

execution_time = @elapsed x, obj_value = revised_simplex(lp)
println("\nExecution time: $execution_time seconds")

println("\nFinal solution:")
for (var, val) in zip(lp.vars, x[1:length(lp.vars)])
    println("$var = $(round(val, digits=6))")
end
println("Objective value: $(round(obj_value, digits=6))")

Initial problem:
c = [3.0, 2.0]
A = [1.0 1.0; 1.0 0.0; 0.0 1.0; -1.0 0.0; 0.0 -1.0]
b = [4.0, 2.0, 3.0, -2.0, -3.0]
Initial basis: [3, 4, 5, 6, 7]
Initial non-basic variables: [1, 2]

Iteration 1
Basic solution: [4.0, 2.0, 3.0, -2.0, -3.0]
Reduced costs: [3.0, 2.0]
Entering variable: 1
Direction: [1.0, 1.0, 0.0, -1.0, 0.0]
Leaving variable: 4
New basis: [3, 1, 5, 6, 7]
New non-basic variables: [4, 2]

Iteration 2
Basic solution: [2.0, 2.0, 3.0, 0.0, -3.0]
Reduced costs: [-3.0, 2.0]
Entering variable: 2
Direction: [1.0, 0.0, 1.0, 0.0, -1.0]
Leaving variable: 3
New basis: [2, 1, 5, 6, 7]
New non-basic variables: [4, 3]

Iteration 3
Basic solution: [2.0, 2.0, 1.0, 0.0, -1.0]
Reduced costs: [-1.0, -2.0]
Optimal solution found

Execution time: 0.00461975 seconds

Final solution:
x1 = 2.0
x2 = 2.0
Objective value: 10.0
